In [ ]:
import pickle
import numpy as np
import pandas as pd
import torch
from torch import nn
import random
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm
import os
import matplotlib.pyplot as plt
import yfinance as yf
from empyrical import sharpe_ratio
from sklearn.preprocessing import LabelEncoder
from src.features import DeepMomentumFeatures, MACDFeatures, DatetimeFeatures, DefaultFeatureCreator
from src.data import MultivariateTrainValTestSplitter
from src.utils import reg_l1, reg_turnover, sharpe_loss
from src.simple_models import *
from src.tft import TFT

In [ ]:
# Train models as described in https://arxiv.org/pdf/2302.10175.pdf
# Each asset is used as feature, data shape is (asset_length, lookback_window, n_assets*n_features_per_asset)

In [ ]:
torch.__version__

In [ ]:
with open('sectors_dict.pickle', 'rb') as f:
    sectors = pickle.load(f)

In [ ]:
# randomly select assets to use

In [ ]:
SEED = 42
N_ASSETS_PER_SECTOR = 5
LENGTH_THRESHOLD = 3000
DATASET_DIRNAME = 'yf_data'
USE_ADJUSTED_CLOSE = True

In [ ]:
np.random.seed(SEED)
all_dfs = []
all_used_assets = []
for current_sector in sectors.keys():
    dfs = []
    assets = []
    for asset in sectors[current_sector]:
        df = pd.read_csv(os.path.join(DATASET_DIRNAME, f'{current_sector}', f'{asset}.csv'))
        # use only stocks with long enough history
        if len(df) > LENGTH_THRESHOLD:
            df['Date'] = pd.to_datetime(df['Date'])
            
            close_col = 'Adj Close' if USE_ADJUSTED_CLOSE else 'Close'
            
            cols = ['Date', 'Open', 'High', 'Low', close_col, 'Volume']
            
            df = df[cols]
            
            df.columns = ['Date'] + ['{}_{}'.format(asset, name) for name in \
                                     ['open', 'high', 'low', 'close', 'volume']]
            dfs.append(df)
            assets.append(asset)

    df = dfs[0]
    for i in range(1, len(dfs)):
        df = pd.merge(df, dfs[i], left_on='Date', right_on='Date', how='inner')

    used_assets = np.random.choice(assets, N_ASSETS_PER_SECTOR, replace=False)
    df = df[['Date'] + [name for name in df.columns[1:] if name.split('_')[0] in used_assets]]
    all_dfs.append(df)
    all_used_assets.extend(list(used_assets))

df = all_dfs[0]
for i in range(1, len(all_dfs)):
    df = pd.merge(df, all_dfs[i], left_on='Date', right_on='Date', how='inner')
df = df.set_index(df['Date'])
print('loaded dataframe shape', df.shape)

In [ ]:
df

In [ ]:
# create features

In [ ]:
USE_DATETIME_FEATURES = False #categorical variables

In [ ]:
used_features = [DeepMomentumFeatures, MACDFeatures] # one can implement and add other features like VWAP, RSI etc
features_configs = [{}, {}]
if USE_DATETIME_FEATURES:
    used_features.append(DatetimeFeatures)
    features_configs.append({})

fc = DefaultFeatureCreator(df, all_used_assets, used_features, features_configs)

features = fc.create_features()


In [ ]:
# check if something is wrong with data

In [ ]:
BAD_VALUES_THRESHOLD = 10000
for key in features.keys():
    assert features[key].isnull().sum().sum() == 0
    assert features[key].max().max() < BAD_VALUES_THRESHOLD
    assert features[key].min().min() > -BAD_VALUES_THRESHOLD

In [ ]:
cols_to_use = ['norm_daily_return',
               'norm_monthly_return',
               'norm_quarterly_return',
               'norm_biannual_return',
               'norm_annual_return',
               'macd_8_24',
               'macd_16_48',
               'macd_32_96']

if USE_DATETIME_FEATURES:
    datetime_cols = ['day_of_week', 'day_of_month', 'month_of_year']
    
    # encode categorical features
    for col in datetime_cols:
        if key in features.keys():
            features[key][col] = LabelEncoder().fit_transform(features[key][col])
else:
    datetime_cols = []

In [ ]:
# available models
MODEL_MAPPING = {'lstm': LSTMnet,
                 'slp': SLP,
                 'mlp': MLP,
                 'conv': TCN,
                 'tft': TFT}

In [ ]:
# configure experiment

In [ ]:
scaling = None #'standard', 'minmax'
model_type = 'mlp' # model type from MODEL_MAPPING
if model_type == 'tft':
    history_size = 63
    encoder_length = 42 # in case of tft encoder length should be int
    model_params = {'device': 'cuda'}
else:
    history_size = 21
    encoder_length = None # in case of non tft model encoder length should be None
    model_params = {}
    
# training is done via expanding window approach:
# train, val, test = (2010-2017, 2017-2018, 2018-2019), then (2010-2018, 2018-2019, 2019-2020) etc  
val_delta = pd.Timedelta('365days')
test_delta = pd.Timedelta('365days')
date_range = pd.date_range('2017-01-01', '2023-12-31', freq='365d')

apply_turnover_reg = False
apply_l1_reg = False
weight_decay = 1e-5
lr = 1e-3
decay_steps = 10
decay_gamma = 0.75
early_stopping_rounds = 10
n_epochs = 2
device = 'cuda'
target_vol = 0.15 #measure for turnover evaluation
basis_points = [0, 1, 5, 10] #coefficients for turnover evaluation

In [ ]:
def _set_seed(seed):

    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [ ]:
for seed in [42]:
    
    if not os.path.exists('weights'):
        os.mkdir('weights')
    if not os.path.exists('results'):
        os.mkdir('results')
    
    test_dts = []
    results = {}
    splitter = MultivariateTrainValTestSplitter(features, cols_to_use, datetime_cols,
                                                'target_returns', 'target_returns_nonscaled',
                                                'daily_vol', scaling=scaling, timesteps=history_size,
                                                 encoder_length=encoder_length)
    for start in date_range:
        train_loader, val_loader, test_loader, test_dt, cat_info = splitter.split(start, val_delta,
                                                                                  test_delta, seed)
        test_dts.append(test_dt)
        if len(test_loader) == 0:
            continue
        
        dt = start
        results[dt] = {}
        
        # in case of tft number of model inputs is different from other model types
        batch_data = next(iter(train_loader))
        if model_type != 'tft':
            batch_x, batch_y, _, _ = batch_data
        else:
            batch_x, _, _, _, _, _, batch_y, _, _ = batch_data
        
        input_dim = batch_x.shape[2]
        output_dim = batch_y.shape[2]
        timesteps = history_size
        
        # fix weights initialization for reproducibility
        _set_seed(seed)
        model = MODEL_MAPPING[model_type](input_dim, output_dim,
                                          timesteps, cat_info=cat_info, **model_params).to(device)
        
        opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        sc = torch.optim.lr_scheduler.StepLR(opt, decay_steps, decay_gamma)

        counter = 0
        
        train_losses = []
        train_l1_losses = []
        train_turnover_losses = []
        val_losses = []
        val_turnover_losses = []
        best_val_sharpe = np.NINF
        
        for e in tqdm(range(n_epochs)):
            train_loss = 0
            train_l1_loss = 0
            train_turnover_loss = 0
            model.train()
            for batch_data in tqdm(train_loader):
                # unpack batches
                for i in range(len(batch_data)):
                    batch_data[i] = batch_data[i].to(device)
                
                if model_type != 'tft':
                    batch_x, batch_y, batch_y_orig, batch_vol = batch_data
                    input_data = [batch_x]
                else:
                    batch_x_enc_real, batch_x_enc_cat, batch_x_dec_real, batch_x_dec_cat, \
                    batch_enc_len, batch_dec_len, batch_y, batch_y_orig, batch_vol = batch_data
                    input_data = [batch_x_enc_real, batch_x_enc_cat, batch_x_dec_real, batch_x_dec_cat, \
                                  batch_enc_len, batch_dec_len]
                # train step
                opt.zero_grad()

                output = model(*input_data)

                l = sharpe_loss(output, batch_y)
                train_loss += l.item()
                
                # optionally apply regularization
                if apply_l1_reg:
                    l_l1 = reg_l1(model)
                    train_l1_loss += l_l1.item()
                    l += l_l1

                if apply_turnover_reg:
                    l_turnover = reg_turnover(output, batch_vol)
                    train_turnover_loss += l_turnover.item()
                    l += l_turnover
                
                l.backward()
                opt.step()
            
            # we do not want learning rate to be too small
            if sc.get_last_lr()[0] > 1e-5:
                sc.step()
            
            val_loss = 0
            val_turnover_loss = 0
            
            preds = []
            returns = []
            vols = []
            
            model.eval()
            
            #evaluate model performance on validation dataset
            with torch.no_grad():
                for batch_data in tqdm(val_loader):
                    # unpack batches
                    for i in range(len(batch_data)):
                        batch_data[i] = batch_data[i].to(device)

                    if model_type != 'tft':
                        batch_x, batch_y, batch_y_orig, batch_vol = batch_data
                        input_data = [batch_x]
                    else:
                        batch_x_enc_real, batch_x_enc_cat, batch_x_dec_real, batch_x_dec_cat, \
                        batch_enc_len, batch_dec_len, batch_y, batch_y_orig, batch_vol = batch_data
                        input_data = [batch_x_enc_real, batch_x_enc_cat, batch_x_dec_real, batch_x_dec_cat, \
                                      batch_enc_len, batch_dec_len]
                    
                    output = model(*input_data)
                    
                    l = sharpe_loss(output, batch_y)
                    val_loss += l.item()

                    if apply_turnover_reg:
                        l_turnover = reg_turnover(output, batch_vol)
                        val_turnover_loss += l_turnover.item()
                    
                    # select last timestep as we no longer need for time axis in batch, collect data
                    returns.append(batch_y[:, -1, :].detach().cpu().numpy())
                    preds.append(output[:, -1, :].detach().cpu().numpy())
                    vols.append(batch_vol[:, -1, :].detach().cpu().numpy())
                    
            # create tensors from collected batches of data
            preds = np.concatenate(preds)
            returns = np.concatenate(returns)
            vols = np.concatenate(vols)
            
            #annualized volatility
            vols = vols * 252**0.5
            # validation turnover
            T = target_vol*np.abs(np.diff(preds/(vols+1e-12), prepend=0.0, axis=0))
            # validation sharpe ratios with different turnover strength
            val_sharpes = {}
            
            # calculate Sharpe Ratio given different turnover strength
            for c in basis_points:
                captured = returns*preds - 1e-4*c*T
                R = np.mean(captured, axis=1)
                sharpes = sharpe_ratio(R)
                sharpes = np.mean(sharpes)
                val_sharpes[c] = sharpes
                
            # one can use sharpe ratio averaged by all turnover coefficients as validation performance metric
            #val_sharpe = np.mean(list(val_sharpes.values()))
            
            #select "pure" Sharpe Ratio as current epoch metric
            val_sharpe = val_sharpes[0]
            
            # if current metric is best, save model weights
            if best_val_sharpe < val_sharpe and e > 0:
                best_val_sharpe = val_sharpe
                counter = 0
                torch.save(model.state_dict(), os.path.join('weights', '{}_seed_{}.pt'.format(model_type, seed)))
            
            else:
                counter += 1
            
            # if metric value didn't improve for several epochs, stop training
            if counter > early_stopping_rounds:
                break
            
            # aggregate losses and metrics, print current epoch training state
            train_loss /= len(train_loader)
            train_l1_loss/= len(train_loader)
            train_turnover_loss /= len(train_loader)
            val_loss /= len(val_loader)
            val_turnover_loss /= len(val_loader)
            
            train_losses.append(train_loss)
            train_l1_losses.append(train_l1_loss)
            train_turnover_losses.append(train_turnover_loss)
            val_losses.append(val_loss)
            val_turnover_losses.append(val_turnover_loss)
            
            print('Iter: ', e)
            print('Train loss: ', round(train_losses[-1], 3))
            print('Val loss: ', round(val_losses[-1], 3))
            print('Validation Sharpe Ratio')
            for key in val_sharpes.keys():
                print('C: ', key, 'SR: ', round(val_sharpes[key], 3))
            if apply_l1_reg:
                print('L1 loss', round(train_l1_losses[-1], 5))
            if apply_turnover_reg:
                print('Train turnover loss: ', round(train_turnover_losses[-1], 5))
                print('Val turnover loss: ', round(val_turnover_losses[-1], 5))
            print('Epochs till end: ', early_stopping_rounds - counter + 1)
            print()
        
        # plot losses evolution during training
        print('Validation dates: ', start, start+val_delta)
        
        plt.figure(figsize=(20, 10))
        plt.title('Loss evolution')
        #plt.plot(train_losses, label='train', marker='o')
        plt.plot(val_losses, label='validation', marker='o')
        plt.ylabel('Loss')
        plt.xlabel('Epochs')
        plt.legend()
        plt.show()
        
        if apply_l1_reg:
            plt.figure(figsize=(20, 10))
            plt.title('L1 regularization loss evolution')
            plt.plot(train_l1_losses, label='train', marker='o')
            plt.ylabel('L1 Loss')
            plt.xlabel('Epochs')
            plt.legend()
            plt.show()
        
        if apply_turnover_reg:
            plt.figure(figsize=(20, 10))
            plt.title('Turnover loss evolution')
            plt.plot(train_turnover_losses, label='train', marker='o')
            plt.plot(val_turnover_losses, label='validation', marker='o')
            plt.ylabel('Turnover loss')
            plt.xlabel('Epochs')
            plt.legend()
            plt.show()
        
        # load best checkpoint in terms of sharpe ratio on validation dataset
        model.load_state_dict(torch.load(os.path.join('weights', '{}_seed_{}.pt'.format(model_type, seed))))
        model = model.to(device)
        model.eval()
        
        val_preds = []
        val_returns = []
        val_returns_orig = []
        val_vols = []

        model.eval()
        
        # calculate model predictions on validation and test datasets
        with torch.no_grad():
            for batch_data in val_loader:
                for i in range(len(batch_data)):
                    batch_data[i] = batch_data[i].to(device)
                
                if model_type != 'tft':
                    batch_x, batch_y, batch_y_orig, batch_vol = batch_data
                    input_data = [batch_x]
                else:
                    batch_x_enc_real, batch_x_enc_cat, batch_x_dec_real, batch_x_dec_cat, \
                    batch_enc_len, batch_dec_len, batch_y, batch_y_orig, batch_vol = batch_data
                    input_data = [batch_x_enc_real, batch_x_enc_cat, batch_x_dec_real, batch_x_dec_cat, \
                                  batch_enc_len, batch_dec_len]

                output = model(*input_data)


                # select last timestep as we no longer need for time axis in batch
                val_returns.append(batch_y[:, -1, :].detach().cpu().numpy())
                val_returns_orig.append(batch_y_orig[:, -1, :].detach().cpu().numpy())
                val_preds.append(output[:, -1, :].detach().cpu().numpy())
                val_vols.append(batch_vol[:, -1, :].detach().cpu().numpy())


        val_preds = np.concatenate(val_preds)
        val_returns = np.concatenate(val_returns)
        # one can evaluate model performance using returns which were not scaled by volatility
        val_returns_orig = np.concatenate(val_returns_orig) 
        val_vols = np.concatenate(val_vols)
        
        test_preds = []
        test_returns = []
        test_returns_orig = []
        test_vols = []

        with torch.no_grad():
            for batch_data in test_loader:
                for i in range(len(batch_data)):
                    batch_data[i] = batch_data[i].to(device)
                
                if model_type != 'tft':
                    batch_x, batch_y, batch_y_orig, batch_vol = batch_data
                    input_data = [batch_x]
                else:
                    batch_x_enc_real, batch_x_enc_cat, batch_x_dec_real, batch_x_dec_cat, \
                    batch_enc_len, batch_dec_len, batch_y, batch_y_orig, batch_vol = batch_data
                    input_data = [batch_x_enc_real, batch_x_enc_cat, batch_x_dec_real, batch_x_dec_cat, \
                                  batch_enc_len, batch_dec_len]

                output = model(*input_data)


                # select last timestep as we no longer need for time axis in batch
                test_returns.append(batch_y[:, -1, :].detach().cpu().numpy())
                test_returns_orig.append(batch_y_orig[:, -1, :].detach().cpu().numpy())
                test_preds.append(output[:, -1, :].detach().cpu().numpy())
                test_vols.append(batch_vol[:, -1, :].detach().cpu().numpy())


        test_preds = np.concatenate(test_preds)
        test_returns = np.concatenate(test_returns)
        # one can evaluate model performance using returns which were not scaled by volatility
        test_returns_orig = np.concatenate(test_returns_orig)
        test_vols = np.concatenate(test_vols)
        
        # collect predictions and data for current window
        
        results[dt]['val'] = {}
        results[dt]['test'] = {}
        results[dt]['test_dt'] = test_dt
        
        results[dt]['val']['preds'] = val_preds
        results[dt]['val']['returns'] = val_returns
        results[dt]['val']['returns_orig'] = val_returns_orig
        results[dt]['val']['vols'] = val_vols
        
        results[dt]['test']['preds'] = test_preds
        results[dt]['test']['returns'] = test_returns
        results[dt]['test']['returns_orig'] = test_returns_orig
        results[dt]['test']['vols'] = test_vols
        
    # dump results of experiment
    with open(os.path.join('results', '{}_seed_{}.pickle'.format(model_type, seed)), 'wb') as f:
        pickle.dump(results, f)